# TAIS - MinC
![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaX3LNhGcAe1HnPZSuWS0oH6af0LJHXcH7If1sQgLCFAT1chNGFg)

This notebook is going to be the basis for my workshop at the PyData 2018 Amsterdam workshop. If you have any questions, please let me know!

You'll build a relatively simple bot, that just asks you about your mood and tries to cheer you up if you're feeling a bit down. 

The tutorial consists of three parts:


*   Part 0: Installation and preparations
*   Part 1: You'll start with a basic bot that can only understand natural language but no dialogues.
*   Part 2: You'll add the abilitiy to understand multiturn dialogues.
*   Part 3: I'll give you further resources so you can extend this simple demo.


## Part 0: Installation

### Let's start with jupyter configuration

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Installation of Rasa
First you'll have to install Rasa Core and NLU in this notebook if you already have them installed in your env, you can just skip this. 

In [2]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -r requirements.txt


  Using cached https://files.pythonhosted.org/packages/ee/3d/b25d35a9f0d381dd1c02d8e04b37c353caaaff4bc32150328eeebe4931f5/prompt_toolkit-1.0.14-py3-none-any.whl


jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.14 which is incompatible.
ipython 7.1.1 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.14 which is incompatible.
  Found existing installation: prompt-toolkit 2.0.7
    Uninstalling prompt-toolkit-2.0.7:
      Successfully uninstalled prompt-toolkit-2.0.7


Let's check if the correct versions are installed (should be rasa_nlu: 0.12.3 rasa_core: 0.9.0a7) and the spacy model is available:

In [3]:
import rasa_nlu
import rasa_core

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))


rasa_nlu: 0.13.5 rasa_core: 0.11.12


### Some additional Tools needed
To do some of the visualizations you will also need graphviz. If you don't have graphviz installed, and this doesn't work: don't worry. I'll show you the graph and besides that visualization everything else will work.

Try installing with anyone of these (or adapt to your operating system):

In [4]:
!{python} -m pip install graphviz;

### Train the Rasa NLU Model

You're going to train a model to recognise these intents, so that when you send a message like "hello" to your bot, it will recognise this as a `"greet"` intent.

In [5]:
import os

from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer
from rasa_nlu import config


intents_directory = '../../bot/data/intents/'

intents = {}

for intent_file in os.listdir(intents_directory):
    intent_file_path = os.path.join(intents_directory, intent_file)

    intents[intent_file] = {}

    intent_list = []
    intent_name = None

    with open(intent_file_path) as f:
        lines = f.readlines()

        for line in lines:
            line = line.strip()

            if line.startswith('##'):
                if intent_name is not None:
                    intents[intent_file][intent_name] = intent_list
                intent_name = line.replace('## intent:', '') 
                intent_list = []

            elif line.startswith('- '):
                intent_list.append(line.replace('- ', ''))


# loading the nlu training samples
training_data = load_data(intents_directory)

# trainer to educate our pipeline
trainer = Trainer(config.load("../../bot/nlu_config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/processo.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 347 (14 distinct intents)
	- Found intents: 'processo_aprovacao', 'processo_preenchimento', 'processo', 'processo_definicao_etapas', 'processo_prazo', 'processo_periodo', 'processo_prazo_envio_cnae', 'processo_admissibilidade', 'processo_reativacao_de_proposta', 'processo_como_funciona', 'processo_situacao_processo', 'processo_execucao', 'processo_analise_de_resultados', 'processo_prazo_desistir_recurso'
	- entity examples: 257 (1 distinct entities)
	- found entities: 'situacao_processo'

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/general.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 131 (8 distinct intents)
	- Found intents: 'diga_mais', 'afirmar_curiosidades', 'afirmar_projeto', 'out_of_scope', 'despedir', 'cumprimen

# Part 2: Adding dialogue capabilities

### Pro Tip: Visualising the Training Data

You can visualise the stories to get a sense of how the conversations go. This is usually a good way to see if there are any stories which don't make sense


In [6]:
from IPython.display import Image
from rasa_core.agent import Agent

agent = Agent('../../bot/domain.yml')

agent.visualize("../../bot/data/stories/definicoes.md", "story_graph_definicoes.png", max_history=2)
agent.visualize("../../bot/data/stories/incentivo.md", "story_graph_incentivo.png", max_history=2)
agent.visualize("../../bot/data/stories/leirouanet.md", "story_graph_leirouanet.png", max_history=2)
agent.visualize("../../bot/data/stories/main.md", "story_graph_main.png", max_history=2)
agent.visualize("../../bot/data/stories/processo.md", "story_graph_processo.png", max_history=2)
agent.visualize("../../bot/data/stories/salic.md", "story_graph_salic.png", max_history=2)

print("Imagens salvas")

INFO:apscheduler.scheduler:Scheduler started
/home/gabibs/.pyenv/versions/3.6.6/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks: 100%|██████████| 15/15 [00:00<00:00, 261.69it/s, # trackers=1]


Imagens salvas


### Training your Dialogue Model

Now comes the fun part! We're going to show Rasa Core the stories we wrote above, and train a model on these examples. 
In this case, the model is a neural network implemented in Keras which learns to predict which action to take next. 

In [7]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.6)

agent = Agent('../../bot/domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('../../bot/data/stories')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=400
)

agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 125/125 [00:01<00:00, 71.71it/s, # trackers=20]
Processed actions: 17179it [00:38, 451.94it/s, # examples=16955]
INFO:rasa_core.policies.keras_policy:Fitting model with 17179 total samples and a validation split of 0.0


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 259)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                37376     
_________________________________________________________________
dense (Dense)                (None, 125)               4125      
_________________________________________________________________
activation (Activation)      (None, 125)               0         
Total params: 41,501
Trainable params: 41,501
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
17179/17179 [==============================] - 6s 343us/step - loss: 2.8971 - acc: 0.3798
Epoch 2/400
17179/17179 [==============================] - 4s 261us/step - loss: 2.1666 - acc: 0.5259
Epoch 3/400
17179/17179 [==============================] - 4s 257u

KeyboardInterrupt: 

### Starting up the bot (with NLU)

Now that we've trained the dialogue **and** language understanding models and saved them, we can start up an `Agent` which will handle conversations for us. 

In [ ]:
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

### Talking to the Bot (with NLU)

We can start talking to the bot in natural language.

In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])


### Evaluation of the dialogue model
As with the NLU model, instead of just subjectively testing the model, we can also evaluate the model on a dataset. You'll be using the training data set again, but usually you'd use a test data set separate from the training data.

In [ ]:
from rasa_core.evaluate import run_story_evaluation

run_story_evaluation("stories.md", "models/dialogue", 
                     nlu_model_path=None, 
                     max_stories=None, 
                     out_file_plot="story_eval.pdf")

### Interactive learning
Unfortunately, this doesn't work in jupyter yet. Hence, we going to do this on the command line. [The repository](https://github.com/tmbo/rasa-demo-pydata18)  contains a makefile with the instructions to run the interactive learning. So go ahead and run `make interactive` on the commandline (make sure your shell is in the righht virtual env).

# Part 3: Next steps

Great job! You've built your first bot that uses Machine Learning to manage dialogues. As a next step, we'd suggest you start building your own bot. Here are a few useful links:



*   Install the Rasa Stack on your machine [here](https://core.rasa.ai/installation.html)
*   Checkout more of the Docs: [NLU](https://nlu.rasa.com) and [Core](https://core.rasa.com)
*   Connect to the community in our [Gitter Chat](https://gitter.im/RasaHQ/rasa_core)



---




*Any feedback for this tutorial?* Please shoot me a mail at tom@rasa.com